In [1]:
using LinearAlgebra

include("./MPS.jl")
using .MPSforQuantum

# MPS

### Left normalized

In [6]:
N = 20
#C0 = normalize!(rand(ComplexF64, 2^N))
C0 = zeros(ComplexF64, 2^N)
C0[1] = 1 # '000'

#D = 10
eps = 1e-3
mps = MPS(C0, eps) # convert to MPS
mps_size(mps) # print the size of MPS

n = 0 # check the state |bin(n)>
println("Restore from MPS: ", restore(mps, n))
println("Original state: ", C0[n + 1])

array 1's size: (1, 1)
array 2's size: (1, 1)
array 3's size: (1, 1)
array 4's size: (1, 1)
array 5's size: (1, 1)
array 6's size: (1, 1)
array 7's size: (1, 1)
array 8's size: (1, 1)
array 9's size: (1, 1)
array 10's size: (1, 1)
array 11's size: (1, 1)
array 12's size: (1, 1)
array 13's size: (1, 1)
array 14's size: (1, 1)
array 15's size: (1, 1)
array 16's size: (1, 1)
array 17's size: (1, 1)
array 18's size: (1, 1)
array 19's size: (1, 1)
array 20's size: (1, 1)
Num of parameters: 40
2^N: 1048576
Restore from MPS: 1.0 + 0.0im
Original state: 1.0 + 0.0im


### Inner product between same states is Identity matrix

In [3]:
I_ = mps[3][1]' * mps[3][1] + mps[3][2]' * mps[3][2]
diag(I_)

8-element Array{Complex{Float64},1}:
 0.9999999999999998 + 0.0im
 0.9999999999999996 + 0.0im
 0.9999999999999998 + 0.0im
 0.9999999999999999 + 0.0im
 0.9999999999999993 + 0.0im
 0.9999999999999998 + 0.0im
 0.9999999999999991 + 0.0im
 0.9999999999999991 + 0.0im

## 簡単な例

In [4]:
tmp = svd(reshape([1, 2, 3, 4, 5, 6, 7, 8], 2, 4))
A1 = tmp.U
C1 = diagm(tmp.S) * tmp.Vt
println(transpose(tmp.U[1, :]) * diagm(tmp.S) * tmp.Vt)
println(transpose(tmp.U[2, :]) * diagm(tmp.S) * tmp.Vt)

[1.0000000000000007 3.0 5.000000000000002 7.000000000000001]
[2.000000000000006 4.0000000000000036 6.000000000000005 8.000000000000007]


In [5]:
tmp = svd(reshape(C1, 4, 2))
A2 = tmp.U
A3 = diagm(tmp.S) * tmp.Vt

2×2 Array{Float64,2}:
  5.3519   13.1824
 -1.16498   0.472968

In [6]:
println(A1)
println(A2)
println(A3)

[-0.6414230279950722 -0.7671873950721774; -0.7671873950721774 0.6414230279950723]
[-0.5661645853690824 -0.7332805685413178; 0.014454499999906334 -0.3762291024141132; -0.8241208487575695 0.49992875081918764; -0.00857649967705636 -0.2661249976998753]
[5.351898726900281 13.182423969136662; -1.1649806938328169 0.4729675442683846]


In [7]:
transpose(A1[1, :]) * A2[1:2, :] * transpose(A3)[1, :] # '000'

1.0000000000000002

# MPO

### $Z_1 Z_2 + Z_2 Z_3$

In [8]:
function dstack(A::Array{ComplexF64,2}, B::Array{ComplexF64,2})
    return cat(A, B, dims = 3)
end

function ddstack(A::Array{ComplexF64,2}, B::Array{ComplexF64,2}, 
                C::Array{ComplexF64,2}, D::Array{ComplexF64,2})
    AC = cat(A, C, dims = 3)
    BD = cat(B, D, dims = 3)
    return cat(AC, BD, dims = 4)
end

ddstack (generic function with 1 method)

In [300]:
pauliZ = convert(Array{ComplexF64,2}, [1 0; 0 -1])
pauliI = convert(Array{ComplexF64,2}, [1 0; 0 1])
zero = convert(Array{ComplexF64,2}, [0 0; 0 0])

O = []
push!(O, dstack(pauliZ, pauliI))
push!(O, ddstack(pauliZ, zero, zero, pauliZ))
push!(O, dstack(pauliI, pauliZ))
#O = [[pauliZ pauliI] [pauliZ zero;pauliZ zero] [pauliI pauliZ]]

O[1][:, :, 2] # [site], [phys1, phys2, i, j]

2×2 Array{Complex{Float64},2}:
 1.0+0.0im  0.0+0.0im
 0.0+0.0im  1.0+0.0im

# General inner products

In [14]:
Hadamard = convert(Array{ComplexF64,2}, [1 1; 1 -1] ./ sqrt(2))
pauliX = convert(Array{ComplexF64,2}, [0 1; 1 0])

2×2 Array{Complex{Float64},2}:
 0.0+0.0im  1.0+0.0im
 1.0+0.0im  0.0+0.0im

In [323]:
N = 3
C0 = zeros(ComplexF64, 2^N)
C0[1] = 1 # '000'
# C0[8] = 1 # '111'

D = 10
eps = 1e-3
mps1 = MPS(C0, eps)
mps1 = OneQubitGate(mps1, Hadamard, 0)
mps1 = OneQubitGate(mps1, Hadamard, 1)
mps1 = OneQubitGate(mps1, Hadamard, 2)

mps2 = MPS(C0, eps)
mps2 = OneQubitGate(mps2, pauliX, 0)
mps1

array 1's size: (1, 1)(1, 1)
array 2's size: (1, 1)(1, 1)
array 3's size: (1, 1)(1, 1)
Num of parameters: 3
2^N: 8
array 1's size: (1, 1)(1, 1)
array 2's size: (1, 1)(1, 1)
array 3's size: (1, 1)(1, 1)
Num of parameters: 3
2^N: 8


3-element Array{Any,1}:
 Array{Complex{Float64},2}[[0.7071067811865475 + 0.0im], [0.7071067811865475 + 0.0im]]
 Array{Complex{Float64},2}[[0.7071067811865475 + 0.0im], [0.7071067811865475 + 0.0im]]
 Array{Complex{Float64},2}[[0.7071067811865475 + 0.0im], [0.7071067811865475 + 0.0im]]

In [10]:
function inner_product(arrs1::Array{Any, 1}, arrs2::Array{Any, 1})
    N = Int64(size(arrs1)[1])
    ip = arrs1[1][1]' * arrs2[1][1] + arrs1[1][2]' * arrs2[1][2]
    for i = 2:N
        phys0 = arrs1[i][1]' * ip * arrs2[i][1]
        phys1 = arrs1[i][2]' * ip * arrs2[i][2]
        ip = phys0 + phys1
    end
    return ip
end

inner_product (generic function with 1 method)

In [43]:
inner_product(mps1, mps2)

1×1 Array{Complex{Float64},2}:
 0.3535533905932737 + 0.0im

# CNOT

### Bell state

In [29]:
mps1

20-element Array{Any,1}:
 Array{Complex{Float64},2}[[1.0 + 0.0im 0.0 + 0.0im], [0.0 + 0.0im 1.0 + 0.0im]]
 Array{Complex{Float64},2}[[0.7071067811865475 + 0.0im; 0.0 + 0.0im], [0.0 + 0.0im; 0.7071067811865475 + 0.0im]]
 Array{Complex{Float64},2}[[1.0 + 0.0im], [0.0 + 0.0im]]
 Array{Complex{Float64},2}[[1.0 + 0.0im], [0.0 + 0.0im]]
 Array{Complex{Float64},2}[[1.0 + 0.0im], [0.0 + 0.0im]]
 Array{Complex{Float64},2}[[1.0 + 0.0im], [0.0 + 0.0im]]
 Array{Complex{Float64},2}[[1.0 + 0.0im], [0.0 + 0.0im]]
 Array{Complex{Float64},2}[[1.0 + 0.0im], [0.0 + 0.0im]]
 Array{Complex{Float64},2}[[1.0 + 0.0im], [0.0 + 0.0im]]
 Array{Complex{Float64},2}[[1.0 + 0.0im], [0.0 + 0.0im]]
 Array{Complex{Float64},2}[[1.0 + 0.0im], [0.0 + 0.0im]]
 Array{Complex{Float64},2}[[1.0 + 0.0im], [0.0 + 0.0im]]
 Array{Complex{Float64},2}[[1.0 + 0.0im], [0.0 + 0.0im]]
 Array{Complex{Float64},2}[[1.0 + 0.0im], [0.0 + 0.0im]]
 Array{Complex{Float64},2}[[1.0 + 0.0im], [0.0 + 0.0im]]
 Array{Complex{Float64},2}[[1.0 + 0.0im]

In [35]:
Hadamard = convert(Array{ComplexF64,2}, [1 1; 1 -1] ./ sqrt(2))

N = 20
C0 = zeros(ComplexF64, 2^N)
C0[1] = 1 # '000'

D = 10
eps = 1e-3

mps1 = MPS(C0, eps)
println("Initial MPS")
mps1 = OneQubitGate(mps1, Hadamard, 0) # -> '001' & '000'
mps_size(mps1)

arr = CX(mps1, 0, eps)
println("\nBell state MPS")
mps_size(arr)
println()
for i=0:3
    res = restore(arr, i)
    println("|", bitstring(i)[end - 1:end], ">: ",  res)
end

Initial MPS
array 1's size: (1, 1)
array 2's size: (1, 1)
array 3's size: (1, 1)
array 4's size: (1, 1)
array 5's size: (1, 1)
array 6's size: (1, 1)
array 7's size: (1, 1)
array 8's size: (1, 1)
array 9's size: (1, 1)
array 10's size: (1, 1)
array 11's size: (1, 1)
array 12's size: (1, 1)
array 13's size: (1, 1)
array 14's size: (1, 1)
array 15's size: (1, 1)
array 16's size: (1, 1)
array 17's size: (1, 1)
array 18's size: (1, 1)
array 19's size: (1, 1)
array 20's size: (1, 1)
Num of parameters: 40
2^N: 1048576

Bell state MPS
array 1's size: (1, 2)
array 2's size: (2, 1)
array 3's size: (1, 1)
array 4's size: (1, 1)
array 5's size: (1, 1)
array 6's size: (1, 1)
array 7's size: (1, 1)
array 8's size: (1, 1)
array 9's size: (1, 1)
array 10's size: (1, 1)
array 11's size: (1, 1)
array 12's size: (1, 1)
array 13's size: (1, 1)
array 14's size: (1, 1)
array 15's size: (1, 1)
array 16's size: (1, 1)
array 17's size: (1, 1)
array 18's size: (1, 1)
array 19's size: (1, 1)
array 20's size: (1

In [21]:
N = 2
C0 = zeros(ComplexF64, 2^N)
C0[1] = 0.7071067811865475
C0[4] = 0.7071067811865475

eps = 1e-3

mps1 = MPS(C0, eps)
mps_size(mps1)
for i=0:3
    res = restore(mps1, i)
    println("|", bitstring(i)[end - 1:end], ">: ",  res)
end

array 1's size: (1, 2)
array 2's size: (2, 1)
Num of parameters: 8
2^N: 4
|00>: 0.7071067811865475 + 0.0im
|01>: 0.0 + 0.0im
|10>: 0.0 + 0.0im
|11>: 0.7071067811865475 + 0.0im


# Expectation value

In [298]:
# prepare bell state
N = 3
C0 = zeros(ComplexF64, 2^N)
C0[1] = 1 # '000'

D = 10
eps = 1e-3

mps0 = MPS(C0, eps)
mps = OneQubitGate(mps0, Hadamard, 0) # -> '001' & '000'
#mps = OneQubitGate(mps, Hadamard, 1)
mps = OneQubitGate(mps, Hadamard, 2)

mps = CX(mps, 0, eps)

for i=1:2, j=1:2, k=1:2
    println("$(i-1)$(j-1)$(k-1): ", mps[1][k] * mps[2][j] * mps[3][i]) # 後ろが最上位bit
end

array 1's size: (1, 1)(1, 1)
array 2's size: (1, 1)(1, 1)
array 3's size: (1, 1)(1, 1)
Num of parameters: 3
2^N: 8
000: Complex{Float64}[0.4999999999999999 + 0.0im]
001: Complex{Float64}[0.0 + 0.0im]
010: Complex{Float64}[0.0 + 0.0im]
011: Complex{Float64}[0.4999999999999999 + 0.0im]
100: Complex{Float64}[0.4999999999999999 + 0.0im]
101: Complex{Float64}[0.0 + 0.0im]
110: Complex{Float64}[0.0 + 0.0im]
111: Complex{Float64}[0.4999999999999999 + 0.0im]


In [299]:
# First step
aa0 = size(mps[1][1])[2]
ss = 2
bb0 = size(O[1][1, 1, :])[1]
arr0_0 = zeros(ComplexF64, ss, bb0, aa0)
for sigma1=1:ss, a2=1:aa0, b = 1:bb0
    for sigma2=1:ss
        arr0_0[sigma1, b, a2] += O[1][sigma1, sigma2, b] * mps[1][sigma2][1, a2]
    end
end
arr0_1 = zeros(ComplexF64, aa0, bb0, aa0)
for a1=1:aa0, a2=1:aa0, b = 1:bb0
    for sigma1=1:ss
        arr0_1[a1, b, a2] += mps[1][sigma1]'[a1, 1] * arr0_0[sigma1, b, a2]
    end
end
println(arr0_1)

# Sencond step
aa1 = size(mps[2][1])[2]
ss = 2
bb1_0, bb1_1 = size(O[2][1, 1, :, :])
arr1_0 = zeros(ComplexF64, ss, aa0, bb1_0, aa1)
for sigma2 = 1:ss, a0_1 = 1:aa0, b1 = 1:bb1_0, a1_0=1:aa1
    for a0_0 = 1:aa0
        arr1_0[sigma2, a0_1, b1, a1_0] += arr0_1[a0_1, b1, a0_0] * mps[2][sigma2][a0_0, a1_0]
    end
end

arr1_1 = zeros(ComplexF64, ss, bb1_1, aa0, aa1)
for sigma1 = 1:ss, b2 = 1:bb1_1, a0_1 = 1:aa0, a1_0=1:aa1
    for sigma2=1:ss, b1 = 1:bb1_0
        arr1_1[sigma1, b2, a0_1, a1_0] += O[2][sigma1, sigma2, b1, b2] * arr1_0[sigma2, a0_1, b1, a1_0]
    end
end

arr1_2 = zeros(ComplexF64, aa1, bb1_1, aa1)
for a1_1=1:aa1, b2 = 1:bb1_1, a1_0=1:aa1
    for sigma1 = 1:ss, a0_1 = 1:aa0
        arr1_2[a1_1, b2, a1_0] += mps[2][sigma1]'[a1_1, a0_1] * arr1_1[sigma1, b2, a0_1, a1_0]
    end
end
println(arr1_2)
    

# Third step
aa2 = size(mps[3][1])[2]
ss = 2
bb2_0 = size(O[3][1, 1, :])
arr2_0 = zeros(ComplexF64, ss, aa1, bb1_1, aa2)
for sigma2 = 1:ss, a1_1=1:aa1, b2 = 1:bb1_1, a2_0 = 1:aa2
    for a1_0=1:aa1
        arr2_0[sigma2, a1_1, b2, a2_0] += arr1_2[a1_1, b2, a1_0] * mps[3][sigma2][a1_0, a2_0]
    end
end

arr2_1 = zeros(ComplexF64, ss, aa1, aa2)
for sigma1 = 1:ss, a1_1=1:aa1, a2_0 = 1:aa2
    for sigma2 = 1:ss, b2 = 1:bb1_1
        arr2_1[sigma1, a1_1, a2_0] += O[3][sigma1, sigma2, b2] * arr2_0[sigma2, a1_1, b2, a2_0]
    end
end

arr2_2 = zeros(ComplexF64, aa2, aa2)
for a2_1 = 1:aa2, a2_0 = 1:aa2
    for sigma1 = 1:ss, a1_1=1:aa1
        arr2_2[a2_1, a2_0] += mps[3][sigma1]'[a2_1, a1_1] * arr2_1[sigma1, a1_1, a2_0]
    end
end
println(arr2_2)

Complex{Float64}[1.0 + 0.0im 1.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im]

Complex{Float64}[0.0 + 0.0im 0.0 + 0.0im; -1.0 + 0.0im 1.0 + 0.0im]
Complex{Float64}[0.9999999999999998 + 0.0im 0.0 + 0.0im]
Complex{Float64}[0.9999999999999997 + 0.0im]


In [245]:
println(size(O[1][1, 1, :]))
println(size(O[2][1, 1, :, :]))
println(size(O[3][1, 1, :]))

(2,)
(2, 2)
(2,)


In [289]:
mps[3][1]

1×1 Array{Complex{Float64},2}:
 0.7071067811865475 + 0.0im

In [303]:
s = bitstring(2)[end - (3):end]

"0010"